In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

In [6]:
# if not os.path.exists('ffmpeg'):
#     !git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg

# !cd ffmpeg && ./configure && make && make install

# ffmpeg_path = os.path.join(os.getcwd(), 'ffmpeg', 'bin')
# os.environ['PATH'] += os.pathsep + ffmpeg_path

'.' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from transformers import pipeline
import torch

MODEL_NAME = "biodatlab/whisper-th-medium-combined"  # specify the model name
lang = "th"  # change to Thai langauge

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(
  language=lang,
  task="transcribe"
)

# pipe = pipeline("automatic-speech-recognition", model=MODEL_NAME, device=device)

In [3]:
import pyaudio
import wave

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

for i in range(0, numdevices):
    if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  Microsoft Sound Mapper - Input
Input Device id  1  -  Microphone (ATR2500x-USB Microp
Input Device id  2  -  Microphone (NVIDIA Broadcast)
Input Device id  3  -  Microphone Array (Realtek(R) Au


In [4]:
devinfo = p.get_device_info_by_index(1)  # Or whatever device you care about.
if p.is_format_supported(44100.0,  # Sample rate
                         input_device=devinfo['index'],
                         input_channels=devinfo['maxInputChannels'],
                         input_format=pyaudio.paInt16):
  print(devinfo['name'] + ' is supported.')

Microphone (ATR2500x-USB Microp is supported.


## Audio Terms

Stack : https://stackoverflow.com/questions/35970282/what-are-chunks-samples-and-frames-when-using-pyaudio

1. "RATE" is the "sampling rate", i.e. the number of frames per second
2. "CHUNK" is the (arbitrarily chosen) number of frames the (potentially very long) signals are split into in this example
3. Yes, each frame will have 2 samples as "CHANNELS=2", but the term "samples" is seldom used in this context (because it is confusing)
4. Yes, size of each sample is 2 bytes (= 16 bits) in this example
5. Yes, size of each frame is 4 bytes
6. Yes, each element of "frames" should be 4096 bytes. `sys.getsizeof()` reports the storage space needed by the Python interpreter, which is typically a bit more than the actual size of the raw data.
7. `RATE * RECORD_SECONDS` is the number of frames that should be recorded. Since the `for` loop is not repeated for each frame but only for each chunk, the number of loops has to be divided by the chunk size `CHUNK`. This has nothing to do with samples, so there is no factor of `2` involved.
8. If you really want to see the hexadecimal values, you can try something like `[hex(x) for x in frames[0]]`. If you want to get the actual 2-byte numbers use the format string `'<H'` with the struct module.

You might be interested in my tutorial about reading WAV files with the wave module, which covers some of your questions in more detail: http://nbviewer.jupyter.org/github/mgeier/python-audio/blob/master/audio-files/audio-files-with-wave.ipynb

In [7]:
p = pyaudio.PyAudio()

MIC_INDEX = 1

SAMPLE_RATE = 44100
CHUNK = 1024
CHANNELS = 2
FORMAT = pyaudio.paInt16

RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "voice/temp.wav"

# Initialize Audio Stream
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=SAMPLE_RATE,
    input=True,
    frames_per_buffer=CHUNK,
    input_device_index=MIC_INDEX,
)

In [8]:
print('Recording...')

frames = []

for i in range(0, int(SAMPLE_RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print('Finished recording.')

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(SAMPLE_RATE)
wf.writeframes(b''.join(frames))
wf.close()

print('Saved to ' + WAVE_OUTPUT_FILENAME)

Recording...
Finished recording.
Saved to voice/temp.wav


In [9]:
text = pipe('voice/temp.wav')['text']
print(text)

สวัสดีครับ test ไมโครโฟน
